# __Проектный практикум.__

## __Описание проекта:__
#### Концепция и прототип продукта, который использует предобученные модели машинного обучения для обработки визуальных данных (изображений) горящих факелов на углеводородных месторождениях с последующей оценкой объема сжигания газа и выбросов СО2 в атмосферу Земли.

#### __Бизнес-постановка задачи:__ Создание автоматизированной системы мониторинга для нефтегазовых компаний и экологических надзорных органов, которая по спутниковым или наземным снимкам месторождений в режиме, близком к реальному времени, обнаруживает факельные установки, оценивает интенсивность горения и переводит её в ключевые бизнес- и экологические метрики: объём бесполезно сожжённого попутного нефтяного газа (ПНГ) и объём выбросов CO₂. _Это позволит компаниям сократить потери ресурсов, оптимизировать производство, выполнять нормативы по утилизации ПНГ и точно отчитываться о выбросах, а регуляторам — объективно контролировать экологическую обстановку._

## __Постановка задачи в терминах Data Science и план решения:__

#### __Задача DS:__ Обнаружить на изображении-обекта инфраструктуры месторождения горящий факел - объект (Object Detection) оценки и реализовать последующую регрессию излеченных из изображения факела признаков (площадь факела, яркость, дымность и пр.) для расчета объемов сжигания углеводородного газа и массу выбросов СО2 в атмосферу Земли.

 - Первичная задача: Обнаружение объектов — модель должна на изображении найти и локализовать (обвести bounding box'ом) все факельные установки.

 - Вторичная задача: Регрессия — на основе характеристик обнаруженного факела (например, площадь и яркость пламени, его геометрические параметры) нужно спрогнозировать числовую величину "в момент": объем сжигаемого газа, а затем и массу выбросов CO₂.

## __Ключевые термины:__

 - __Bounding Box:__ Прямоугольная область, ограничивающая объект (факел) на изображении.

 - __Предобученная модель (Pretrained Model):__ Модель (например, YOLO, которая часто используется в Roboflow), уже обученная на большом общем наборе данных (например, COCO). Это позволяет "дообучить" (fine-tune) её на наших специфичных данных, экономя время и вычислительные ресурсы.

 - __Аннотация данных:__ Процесс разметки данных — на каждом изображении из набора данных вручную обводятся факелы и присваивается класс ("flare").

 - __Перенос обучения (Transfer Learning):__ Использование предобученной модели в качестве стартовой точки для нашей конкретной задачи.

 - __Метрики оценки для детекции:__ Precision (точность), Recall (полнота), mAP (mean Average Precision) — покажут, насколько хорошо модель находит факелы и не путает их с другими объектами.

 - __Метрики для регрессии:__ MAE (Mean Absolute Error), RMSE (Root Mean Square Error) — покажут среднюю ошибку в оценке объема газа (например, в тыс. м³/сутки)

## __Последовательность шагов (План решения задачи):__

### __Этап 1: Подготовка данных (Data Preparation)__

#### 1.1. Сбор данных: Найти и скачать изображения горящих факелов со спутников (Sentinel-2, Landsat) или с дронов. Источники: открытые репозитории (NASA, ESA), Kaggle датасеты, партнеры.
#### 1.2. Аннотирование (разметка): Использовать инструменты вроде Roboflow Annotate или CVAT для разметки изображений. На каждом снимке нужно нарисовать bounding box вокруг каждого факела и присвоить ему класс, например, flare. Это самый трудоемкий, но критически важный этап.
#### 1.3. Предобработка данных (Preprocessing): С помощью Roboflow или вручную:
* Привести все изображения к единому размеру (например, 640x640).
* Применить аугментации (Augmentation) для увеличения разнообразия данных: случайные повороты, изменение яркости/контрастности, добавление шума, имитация облачности. Это поможет модели стать более устойчивой к разным условиям съемки.
* Разделить данные на обучающую (train), валидационную (validation) и тестовую (test) выборки (например, 70%/20%/10%).
---

### __Этап 2: Обучение модели детекции (Model Training for Detection)__

#### 2.1. Выбор и настройка модели: Взять предобученную модель model из Roboflow (вероятно, это YOLOv5/v8). Подготовить конфигурационный файл, указав количество классов (1 — flare), пути к данным и гиперпараметры.

#### 2.2. Дообучение (Fine-tuning): Загрузить размеченный датасет в Roboflow или использовать свой пайплайн. Провести обучение модели на наших данных, используя предобученные веса в качестве стартовых.

#### 2.3. Валидация и оценка: На валидационной выборке отслеживать метрики (mAP@0.5). Анализировать, где модель ошибается (не находит маленькие факелы, принимает за факел солнце и т.д.).

#### 2.4. Тестирование: Финальная проверка качества на тестовой выборке, которую модель никогда не видела. Убедиться, что mAP, Precision и Recall находятся на приемлемом уровне.
---

### __Этап 3: Разработка модуля оценки объема выбросов (Regression Pipeline)__

#### 3.1. Извлечение признаков (Feature Extraction): Для каждого обнаруженного факела (по bounding box) вычислить характеристики из исходного изображения:
* Площадь пламени в пикселях.
* Средняя/максимальная яркость в инфракрасном канале (если есть мультиспектральные данные).
* Отноше ние сторон bounding box, его высота.

#### 3.2. Калибровка и регрессия:
* Найти открытые исследования или провести калибровку, чтобы связать извлеченные признаки (площадь, яркость) с физическими параметрами (тепловая мощность, объем сжигаемого газа). Возможно, потребуется простой регрессионный модель (линейная, полиномиальная) или lookup-таблица.
* Использовать известный коэффициент пересчета (например, ~2.75 кг CO₂ на 1 м³ сожженного природного газа) для расчета выбросов.
---

### __Этап 4: Интеграция и вывод (Deployment)__

#### 1. Создание пайплайна: Написать скрипт на Python, который:
* Принимает новое изображение.
* Запускает модель детекции.
* Для каждого обнаруженного факела извлекает признаки и запускает регрессионную модель для оценки объема сжигаемого газа и выбросоов CO₂ "в момент".
* Выводит итоговое изображение с обведенными факелами и подписями с рассчитанными значениями.

#### 2. Прототипирование (создание MVP): Создать простой веб-интерфейс (например, на Streamlit/Gradio) или API, куда пользователь может загрузить снимок и получить результат.

## __Код MVP (прототип продукта)__

In [ ]:
import sys
import os
# Установите рабочую директорию
project_root = r"E:\\Python\\MIFI\\project-practice"
os.chdir(project_root)
print("Текущая директория:", os.getcwd())

# Ячейка 2: Запуск основного кода
%run main.py


Текущая директория: E:\Python\MIFI\project-practice
🚀 Загружаю модель YOLO...
✅ Модель загружена: E:\Python\MIFI\project-practice\models\trained\best_model.pt
📊 Классы модели: {0: 'fire', 1: 'smoke'}

🚀 Сервер анализатора газовых факелов
📁 Базовая директория: E:\Python\MIFI\project-practice
🤖 Модель: E:\Python\MIFI\project-practice\models\trained\best_model.pt
📤 Загрузки: E:\Python\MIFI\project-practice\static\uploads
💾 Результаты: E:\Python\MIFI\project-practice\static\results
🌐 Откройте: http://localhost:8000


INFO:     Will watch for changes in these directories: ['E:\\Python\\MIFI\\project-practice']
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)
INFO:     Started reloader process [28352] using StatReload
